In [1]:
import modules
import numpy as np
import importlib
importlib.reload(modules)
from scipy.optimize import minimize


# design variables
x_name = ['x_wec','x_pen']
x_list = modules.variable_lookup(x_name)
x_list_default_values = modules.default_values(x_name)
x0 = []
for i in range(len(x_list)):
    x0.append(x_list_default_values[x_list[i]])

# parameters
wec_types = ['attenuator','terminator','point absorber']

# fill default parameters
all_vars = ['x_wec','x_wec_type','x_pen','p_pen','x_env','p_fish','p_wec']

default_vars = []
for i in range(len(all_vars)):
    if all_vars[i] not in x_name:
        default_vars.append(all_vars[i])

p = modules.default_values(default_vars)

# simulation
J = modules.obj(x0,x_name, p)

#optimization
opt_idx = 0
res={}
for i in range(len(wec_types)):
    p['wec_type'] = wec_types[i]
    
    arguments = (x_name, p)

    cons = ({'type': 'ineq', 'fun': modules.ineq_constraint, 'args': arguments},
            {'type': 'eq', 'fun': modules.eq_constraint, 'args': arguments})

    bnds = tuple((0,1) for x in x0)

    res[i] = minimize(modules.obj, x0, args=arguments, method='SLSQP', constraints=cons)
    #print(res[i], '\n')
    
    if res[i].fun < res[opt_idx].fun:
        opt_idx = i


res[opt_idx]

     fun: array(2.26485211e-05)
     jac: array([0.00000000e+00, 0.00000000e+00, 4.54747351e-13])
 message: 'Optimization terminated successfully'
    nfev: 276
     nit: 66
    njev: 66
  status: 0
 success: True
       x: array([1.00000000e+01, 6.61380437e+07, 5.12393575e+03])